# 모델 저장과 복원
- https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

- 저장 대상: 모델 코드, 가중치, 파라미터
- 저장 시점: 훈련하는 도중이나 훈련이 끝난 후 (모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됨)

In [3]:
!pip install -q pyyaml h5py

In [4]:
import os 
import tensorflow as tf 
from tensorflow import keras 
print(tf.version.VERSION)
print(tf.__version__)

2.5.0-dev20201230
2.5.0-dev20201230


In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000]
test_images = test_images[:1000]

In [8]:
train_images.shape

(1000, 28, 28)

In [13]:
train_images = train_images.reshape(-1, 28*28) / 255.0
test_images = test_images.reshape(-1, 28*28) / 255.0

train_images.shape

(1000, 784)

In [14]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2), # 0.2 비율로 랜덤하게 0으로 변환
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam', 
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  # Computes the crossentropy loss between the labels and predictions.
                  # Use this crossentropy loss function when there are two or more label classes.
                  metrics=['accuracy']
                 )
    return model

model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 체크포인트 콜백
- 훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하는 방법 (모델 재사용하거나 이어서 사용할 수 있도록)
- tf.keras.callbacks.ModelCheckpoint

In [15]:
checkpoint_path = 'training_1/cp.ckpt' # .ckpt (확장자): 텐서플로 체크포인트 포맷
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_dir

'training_1'

In [16]:
# 콜백 정의 (모델 가중치 저장)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels),
          callbacks=[cp_callback]) # 콜백을 훈련에 전달

Epoch 1/10
32/32 [==============================] - 1s 8ms/step - loss: 2.3030 - accuracy: 0.1071 - val_loss: 2.3802 - val_accuracy: 0.1070

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 2.3005 - accuracy: 0.1069 - val_loss: 2.3858 - val_accuracy: 0.0830

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 2.3002 - accuracy: 0.1043 - val_loss: 2.3928 - val_accuracy: 0.0840

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 2.2987 - accuracy: 0.1165 - val_loss: 2.4059 - val_accuracy: 0.0900

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 2.2992 - accuracy: 0.1083 - val_loss: 2.4249 - val_accuracy: 0.0870

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s 

In [19]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


### 가중치만 복원
- 원본 모델과 동일한 구조로 모델을 만들어야 -> 다른 객체이지만 가중치 공유 가능

In [32]:
model = create_model() # compile까지만 진행
loss, acc = model.evaluate(test_images, test_labels, verbose=2) # 레이어 정의 -> compile -> fit -> evaluate

32/32 - 0s - loss: 2.3880 - accuracy: 0.1130


In [33]:
### 체크포인트 콜백 매개변수: 이름 or 체크포인트 주기 

checkpoint_path = 'traning_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True, # 가중치만 저장
    period=5 # 5 에포크마다 가중치 저장
)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

In [34]:
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # model.save_weights(checkpoint_path.format(epoch=N)) 자동으로 처리
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to traning_2/cp-0005.ckpt

Epoch 00010: saving model to traning_2/cp-0010.ckpt

Epoch 00015: saving model to traning_2/cp-0015.ckpt

Epoch 00020: saving model to traning_2/cp-0020.ckpt

Epoch 00025: saving model to traning_2/cp-0025.ckpt

Epoch 00030: saving model to traning_2/cp-0030.ckpt

Epoch 00035: saving model to traning_2/cp-0035.ckpt

Epoch 00040: saving model to traning_2/cp-0040.ckpt

Epoch 00045: saving model to traning_2/cp-0045.ckpt

Epoch 00050: saving model to traning_2/cp-0050.ckpt


In [35]:
!ls {checkpoint_dir} # multiple machine이라면 shard 별로 suffix 다르게 붙었을 것 (00000-of-00001)

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [36]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [43]:
latest # = 'traning_2/cp-0020.ckpt'

'traning_2/cp-0020.ckpt'

In [44]:
model = create_model()
model.load_weights(latest)

In [45]:
loss, acc = model.evaluate(test_images, test_labels, verbose=2) # fit한 weight를 넣었는데 acc 왜 이 모양..?

32/32 - 0s - loss: 2.5209 - accuracy: 0.0720


In [47]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')
loss, acc = model.evaluate(test_images, test_labels, verbose=2) # 위의 acc, loss와 같은 수준

32/32 - 0s - loss: 2.5209 - accuracy: 0.0720


### 전체 모델 
- model.save 메서드: 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장 (이전 코드 없어도 됨)
- 포맷: SavedModel(기본 포맷), HDF5
- 둘다 TensorFlow.js로 모델 로드 후 웹 브라우저에서 모델 훈련/실행 가능. 모바일에서는 TensorFlow Lite 사용 가능

- SavedModel: 모델을 직렬화하는 방법 -> tf.keras.models.load_model로 복원할 수 있고 텐서플로 서빙과 호환됩니다

In [48]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 2.3027 - accuracy: 0.0938
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 2.3011 - accuracy: 0.1206
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 2.3003 - accuracy: 0.1081
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 2.3007 - accuracy: 0.1048
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 2.2995 - accuracy: 0.1102


In [59]:
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 2.4372 - accuracy: 0.0990


In [50]:
!mkdir -p saved_model

In [54]:
# SavedModel
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [56]:
!ls saved_model/my_model

assets            keras_metadata.pb saved_model.pb    variables


In [57]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [58]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2) # 위와 동일 

32/32 - 0s - loss: 2.4372 - accuracy: 0.0990


In [60]:
# HDF5
model.save('my_model.h5')

In [61]:
new_model = tf.keras.models.load_model('my_model.h5')

In [62]:
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [63]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 2.4372 - accuracy: 0.0990


HDF5 (Keras 기본)
- 포함 내용: 가중치 값, 모델 구조, 모델의 훈련 설정(컴파일할 때 전달한 값), 옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작 가능)
- 체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.???? 이해 못함 

- SaveModel과의 차이
    - SavedModel: 실행그래프로 저장, 원본 코드 없이 상속 모델과 사용자 정의 객체 저장 가능 (???)
    - HDF5: 사용자 정의 객체 저장하려면 get_config, from_config 메서드 필요 (?)